# Introduction to Python for Data Engineering

This notebook demonstrates how to read data from a public API, process it to create useful information, and store the result in an SQLite database.

It assumes you have read [this article](https://www.startdataengineering.com/post/python-for-de/#extract--load-read-and-write-data-to-any-system).

**Acknowledgment**: Parts of this notebook were developed with the assistance of ChatGPT, an AI language model by OpenAI, to support instructional design and coding tasks related to data engineering concepts.

In [95]:
# Install required packages
!pip install requests

In [96]:
# Import libraries
import requests
import sqlite3
from datetime import datetime

In [97]:
# Get target currency
target_currency = input("Enter the target currency code (eg: zar, eur, gbp): ").lower()

Enter the target currency code (eg: zar, eur, gbp): zar


In [98]:
# Read data from CoinGecko API
url = f"https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd,{target_currency}"
response = requests.get(url)
data = response.json()

print("API Response:", data)


API Response: {'bitcoin': {'usd': 121616, 'zar': 2136605}}


In [99]:
# Validate data
if "bitcoin" not in data or target_currency not in data["bitcoin"]:
    raise ValueError(f"Currency '{target_currency}' not found. API may have rate-limited you or currency is invalid.")

usd_price = data["bitcoin"]["usd"]
target_price = data["bitcoin"][target_currency]
timestamp = datetime.now().isoformat()

print(f"Bitcoin price in USD: {usd_price}")
print(f"Bitcoin price in {target_currency.upper()}: {target_price}")

Bitcoin price in USD: 121616
Bitcoin price in ZAR: 2136605


In [100]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS prices (
    -- These are SQL comments. Similarly /* ... */ for multi-line comments
    -- If timestamp should be unique (e.g., one price per moment):
    -- timestamp TEXT PRIMARY KEY,
    -- If multiple prices per second are expected, or if you want a simple unique ID:
    -- id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    usd REAL,
    zar REAL
)
''')

# Insert the new row
cursor.execute("INSERT INTO prices VALUES (?, ?, ?, ?)", (timestamp, usd_price, target_currency.upper(), target_price))
conn.commit()
conn.close()


In [101]:
# Check if data was successfully written
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Show the last 5 rows
rows = cursor.execute("SELECT * FROM prices ORDER BY timestamp DESC LIMIT 5").fetchall()
for row in rows:
    print(row)

conn.close()


('2025-08-14T10:08:37.093494', 121616.0, 'ZAR', 2136605.0)
('2025-08-14T09:51:39.426658', 121654.0, 'ZAR', 2136990.0)
('2025-08-14T09:50:19.009495', 121664.0, 'GBP', 89562.0)
('2025-08-14T09:50:04.192224', 121666.0, 'ZAR', 2137223.0)
('2025-08-14T09:49:51.656130', 121680.0, 'EUR', 104134.0)
